In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt


--2023-07-24 00:57:45--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: ‘edgar_allan_poe.txt’

edgar_allan_poe.txt 100%[===================>]  26.00K  --.-KB/s    in 0s      

2023-07-24 00:57:45 (76.8 MB/s) - ‘edgar_allan_poe.txt’ saved [26622/26622]

--2023-07-24 00:57:45--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input= ['edgar_allan_poe.txt','robert_frost.txt']

In [5]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [6]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [11]:
#collect data to list
#removing puntuation and setting everything to lowercase
#rstrip removes new line character
input_text=[]
labels=[]
for label,f in enumerate(input):
  print(f"{f} corresponds to label: {label}")
  for line in open(f):
    line=line.rstrip().lower()
    if line:
      line=line.translate(str.maketrans('','', string.punctuation))
      input_text.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label: 0
robert_frost.txt corresponds to label: 1


In [18]:
train_text, test_text, ytrain, ytest = train_test_split(input_text,labels)

In [19]:
len(ytrain), len(ytest)

(1615, 539)

In [20]:
train_text[:5]

['till well toward noon when the storm settled down',
 'wanderers in that happy valley',
 'but though it runs unbridled off its course',
 'timeeaten towers that tremble not',
 'there open temples open graves']

In [21]:
ytrain[:5]

[1, 0, 1, 0, 0]

In [24]:
idx=1
word2idx={'<unk>':0}  #coverting text to integers

In [26]:
#populate word2idx
for text in train_text:
  tokens=text.split()
  for token in tokens:
    if token not in word2idx:
      word2idx[token]=idx
      idx+=1

In [27]:
word2idx

{'<unk>': 0,
 'till': 1,
 'well': 2,
 'toward': 3,
 'noon': 4,
 'when': 5,
 'the': 6,
 'storm': 7,
 'settled': 8,
 'down': 9,
 'wanderers': 10,
 'in': 11,
 'that': 12,
 'happy': 13,
 'valley': 14,
 'but': 15,
 'though': 16,
 'it': 17,
 'runs': 18,
 'unbridled': 19,
 'off': 20,
 'its': 21,
 'course': 22,
 'timeeaten': 23,
 'towers': 24,
 'tremble': 25,
 'not': 26,
 'there': 27,
 'open': 28,
 'temples': 29,
 'graves': 30,
 'ledges': 31,
 'show': 32,
 'lines': 33,
 'ruled': 34,
 'southeastnorthwest': 35,
 'as': 36,
 'if': 37,
 'by': 38,
 'eye': 39,
 'pairs': 40,
 'out': 41,
 'of': 42,
 'forty': 43,
 'firkins': 44,
 'to': 45,
 'be': 46,
 'a': 47,
 'nova': 48,
 'or': 49,
 'else': 50,
 'some': 51,
 'remoteness': 52,
 'defied': 53,
 'them': 54,
 'nor': 55,
 'ghoulhaunted': 56,
 'woodland': 57,
 'weir': 58,
 'makes': 59,
 'all': 60,
 'worse': 61,
 'you': 62,
 'must': 63,
 'blind': 64,
 'he': 65,
 'never': 66,
 'takes': 67,
 'money': 68,
 'theyre': 69,
 'worth': 70,
 'bells': 71,
 'pipes': 72,


In [33]:
list(word2idx.items())[3]

('toward', 3)

In [34]:
len(word2idx)

2517

In [37]:
#covert data into integer format
train=[]
test=[]

for text in train_text:
  tokens=text.split()
  line_as_int=[word2idx[token] for token in tokens]
  train.append(line_as_int)
for text in test_text:
  tokens=text.split()
  line_as_int=[word2idx.get(token,0) for token in tokens]
  test.append(line_as_int)

In [38]:
train[100:105
    ]

[[156, 47, 380, 329, 36, 42, 47, 381, 382],
 [11, 211, 73, 383],
 [384, 135, 6, 385, 45, 6, 386],
 [387, 388, 194, 389, 390, 391],
 [392, 393, 306, 394, 395, 263]]

In [39]:
#initialize A and pi matrices - for both classes
#using markov model
#initializing with one fot add-one smoothing
v = len(word2idx)
a0=np.ones((v,v))
pi0=np.ones(v)

a1= np.ones((v,v))
pi1=np.ones(v)

In [40]:
#compute counts for A and pi
def compute_counts (text_as_int,a,pi):
  for tokens in text_as_int:
    last_idx=None
    for idx in tokens:
      if last_idx is None:
        #its the first word in a sentence
        pi[idx]+=1
      else:
        #last word, so count a transition
        a[last_idx,idx]+=1
      #update last idx
      last_idx=idx


In [42]:
compute_counts([t for t,y in zip(train,ytrain) if y==0],a0,pi0)
compute_counts([t for t,y in zip(train,ytrain) if y==1],a1,pi1)

In [43]:
#normalize A and pi so they are valid probability matrices
a0/=a0.sum(axis=1,keepdims=True)
pi0/=pi0.sum()

a1/=a1.sum(axis=1,keepdims=True)
pi1/=pi1.sum()

In [44]:
#log A and pi since we don't need the actual probs
loga0=np.log(a0)
logpi0= np.log(pi0)

loga1=np.log(a1)
logpi1= np.log(pi1)

In [45]:
#compute priors

count0=sum(y==0 for y in ytrain)
count1=sum(y==1 for y in ytrain)
total = len(ytrain)
p0=count0/total
p1=count1/total
logp0=np.log(p0)
logp1=np.log(p1)
p0,p1

(0.3430340557275542, 0.6569659442724458)

In [60]:
#build classifier

class classifier:
  def __init__(self,logas,logpis,logpriors):
    self.logas=logas
    self.logpis=logpis
    self.logpriors=logpriors
    self.K=len(logpriors) #number of classes

  def _compute_log_likelihood(self,input_,class_):
    logA=self.logas[class_]
    logpi=self.logpis[class_]

    last_idx=None
    logprob=0
    for idx in input_:
      if last_idx is None:
        #its the first token
        logprob+=logpi[idx]
      else:
        logprob+=logA[last_idx,idx]

      #update idx
      last_idx=idx
    return logprob

  def predict(self,inputs):
    predictions =np.zeros(len(inputs))
    for i,input in enumerate(inputs):
      posterior=  [self._compute_log_likelihood(input,c )+ self.logpriors[c]
                   for c in range(self.K)]
      pred=np.argmax(posterior)
      predictions[i]=pred
    return predictions


In [61]:
#Each array must be in order since classes  are assumed to index these lists
clf = classifier([loga0,loga1],[logpi0,logpi1],[logp0,logp1])

In [62]:
ptrain=clf.predict(train)
print(f"Train acc: {np.mean(ptrain==ytrain)}" )

Train acc: 0.9993808049535604


In [65]:
ptest=clf.predict(test)
print(f"Test acc: {np.mean(ptest==ytest)}")

Test acc: 0.8385899814471243
